# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2: Pandas and Web Scraping with Beautiful Soup

**Harvard University**<br>
**Fall 2019**<br>
**Instructors:** Pavlos Protopapas, Kevin Rader, and Chris Tanner<br>
**Lab Instructors:** Chris Tanner and Eleni Kaxiras <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, Eleni Kaxiras

---

In [16]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

FileNotFoundError: [Errno 2] No such file or directory: '../../styles/cs109.css'

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

*This lab corresponds to lectures #1, #2, and #3 and maps on to homework #1 and further.*

### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [ ]:
#Read the data into a dataframe
df = pd.read_csv("../data/goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df

Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [ ]:
# your code here
df=pd.read_csv("../data/goodreads.csv", header=None,
               names=["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name'],
)

#Examine the first few rows of the dataframe
df.head()

### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [ ]:
# your code here
####### 
df.dtypes
####### 


*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [ ]:
print(df.shape)
df.columns

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [ ]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

In [ ]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [17]:
df[df.year.isnull()]

df.year.isnull()
df.shape

(6000, 10)

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [18]:
#Treat the missing or invalid values in your dataframe
####### 

df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [19]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [20]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5993, 10)

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [21]:
# your code here
df.rating_count=df.rating_count.astype(int)
df.review_count=df.review_count.astype(int)
df.year=df.year.astype(int)

C:\Users\tarla\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [22]:
df.dtypes

rating          float64
review_count      int32
isbn             object
booktype         object
author_url       object
year              int32
genre_urls       object
dir              object
rating_count      int32
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [23]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

C:\Users\tarla\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [24]:
np.sum(df.genre_urls.isnull=="")

0

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [25]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [26]:
test_string.split('.')[-1]

'Suzanne_Collins'

In [27]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'Suzanne_Collins'

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [28]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name = url.split('/')[-1].split('.')[1:][0]
    ####### 
    return name

In [29]:
def extract_author_name(url):
    return url.split('.')[-1]

In [30]:
df['author'] = df.author_url.map(extract_author_name)
df.author[0:5]

<ipython-input-30-e3601a0dd63a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['author'] = df.author_url.map(extract_author_name)


0    Suzanne_Collins
1        J_K_Rowling
2    Stephenie_Meyer
3         Harper_Lee
4        Jane_Austen
Name: author, dtype: object

In [31]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

<ipython-input-31-aa22e8cb8eeb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['author'] = df.author_url.map(get_author)


0    Suzanne_Collins
1        J_K_Rowling
2    Stephenie_Meyer
3         Harper_Lee
4        Jane_Austen
Name: author, dtype: object

<div class="exercise"><b>Exercise</b></div>

Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [32]:
df.genre_urls.head()

0    /genres/young-adult|/genres/science-fiction|/g...
1    /genres/fantasy|/genres/young-adult|/genres/fi...
2    /genres/young-adult|/genres/fantasy|/genres/ro...
3    /genres/classics|/genres/fiction|/genres/histo...
4    /genres/classics|/genres/fiction|/genres/roman...
Name: genre_urls, dtype: object

In [33]:
# your code here
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

young-adult
science-fiction
dystopia
fantasy
science-fiction
romance
adventure
book-club
young-adult
teen
apocalyptic
post-apocalyptic
action


<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [34]:
def split_and_join_genres(url):
    # your code here
    genres=url.strip().split('|')
    genres=[e.split('/')[-1] for e in genres]
    return "|".join(genres)

In [35]:
def sep_genres(gen):
    list_of_genres = [x.split('/')[-1] for x in gen.split('|')]
    return "|".join(list_of_genres)
df['genres'] = df.genre_urls.map(sep_genres)

<ipython-input-35-910861442c91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df.genre_urls.map(sep_genres)


Test your function

In [36]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young-adult|science-fiction'

In [37]:
split_and_join_genres("")

''

<div class="exercise"><b>Exercise</b></div>

Use map again to create a new "genres" column

In [38]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

<ipython-input-38-a1e9cb533342>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres']=df.genre_urls.map(split_and_join_genres)


,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)",Suzanne_Collins,young-adult|science-fiction|dystopia|fantasy|s...
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,fantasy|young-adult|fiction|fantasy|magic|chil...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,young-adult|fantasy|romance|paranormal|vampire...
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,classics|fiction|historical-fiction|academic|s...
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,classics|fiction|romance|historical-fiction|li...


Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [39]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
1014,4.23,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar,historical-fiction|fiction|cultural|france|cla...
5620,4.11,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar,fiction|historical-fiction|cultural|france|eur...


Let us delete the `genre_urls` column.

In [40]:
del df['genre_urls']

And then save the dataframe out!

In [41]:
df.to_csv("../data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [42]:
# your code here
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

47                               The Odyssey
246                    The Iliad/The Odyssey
455                             The Republic
596                               The Aeneid
629                              Oedipus Rex
674                           The Art of War
746                        The Bhagavad Gita
777                                 Antigone
1233                       The Oedipus Cycle
1397                          Aesop's Fables
1398                   The Epic of Gilgamesh
1428                                   Medea
1815                            The Oresteia
1882         The Trial and Death of Socrates
2078    The History of the Peloponnesian War
2527                           The Histories
3133                          Complete Works
3274                  The Nicomachean Ethics
3757                              Lysistrata
4402                           The Symposium
4475                                 Apology
5367                          Five Dialogues
Name: name

We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [43]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [44]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,genres
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1,1
A_J_Salt,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
_,42,42,42,42,42,42,42,42,42,42
_gota_Krist_f,1,1,1,1,1,1,1,1,1,1
_mile_Zola,4,4,4,4,4,4,4,4,4,4


Lots of useless info there. One column should suffice

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense! 

- Create a new column with number of books for each author and find the most prolific author!

In [45]:
dfgb_author = df.copy()

In [93]:
def list_authors_gb(lis):
    return "|".join(lis)

In [94]:
gb_teste = dfgb_author[['rating','name','author']].groupby('author').agg({'rating':min,
                                                                         'name':list_authors_gb})

In [99]:
gb_teste.name[0]
   

'Winnie-the-Pooh|The House at Pooh Corner|The House at Pooh Corner and Now We Are Six|The World of Winnie-the-Pooh|Now We Are Six|The Complete Tales and Poems of Winnie-the-Pooh'

In [47]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
2213,4.19,1169,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2003,dir23/2750180.html,15781,ÙØ§ ØªØ­Ø²Ù,A_id_al_Qarni,religion|religion|islam|self-help|non-fiction|...
5998,3.91,281,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ,A_id_al_Qarni,religion|islam|religion|self-help|spirituality...


In [48]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

b'\xff\xfe\xd9\x00\x84\x00\xd8\x00\xa7\x00 \x00\xd8\x00\xaa\x00\xd8\x00\xad\x00\xd8\x00\xb2\x00\xd9\x00\x86\x00'

In [49]:
# let's examine the columns we have
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'dir', 'rating_count', 'name', 'author', 'genres'], dtype='object')

In [50]:
df[['rating','name','author']].head()

,rating,name,author
0,4.40,"The Hunger Games (The Hunger Games, #1)",Suzanne_Collins
1,4.41,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling
2,3.56,"Twilight (Twilight, #1)",Stephenie_Meyer
3,4.23,To Kill a Mockingbird,Harper_Lee
4,4.23,Pride and Prejudice,Jane_Austen


Create the GroupBy table

In [51]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

In [52]:
authors = authors.reset_index()
authors.head()

,author,rating,name
0,A_A_Milne,4.365,Winnie-the-Pooh|The House at Pooh Corner|The H...
1,A_G_Howard,4.020,"Splintered (Splintered, #1)"
2,A_J_Cronin,4.220,The Keys of the Kingdom
3,A_J_Jacobs,3.750,The Year of Living Biblically
4,A_J_Salt,4.940,Nik Nassa & the Mark of Destiny


In [53]:
authors['name'] = authors.name.astype(str)

In [54]:
authors.name.str.split('|')

0       [Winnie-the-Pooh, The House at Pooh Corner, Th...
1                           [Splintered (Splintered, #1)]
2                               [The Keys of the Kingdom]
3                         [The Year of Living Biblically]
4                       [Nik Nassa & the Mark of Destiny]
                              ...                        
2640    [Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...
2641             [The Notebook, The Proof, The Third Lie]
2642    [Germinal (Les Rougon-Macquart, #13), L'Assomm...
2643                              [Oscar et la dame rose]
2644                            [The Bookseller of Kabul]
Name: name, Length: 2645, dtype: object

In [55]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

,author,rating,name
0,A_A_Milne,4.365,"[Winnie-the-Pooh, The House at Pooh Corner, Th..."
1,A_G_Howard,4.020,"[Splintered (Splintered, #1)]"
2,A_J_Cronin,4.220,[The Keys of the Kingdom]
3,A_J_Jacobs,3.750,[The Year of Living Biblically]
4,A_J_Salt,4.940,[Nik Nassa & the Mark of Destiny]


In [56]:
# count the books - create new column
len(authors.name[0])

6

In [57]:
authors['num_books'] = authors['name'].str.len()
authors

,author,rating,name,num_books
0,A_A_Milne,4.365000,"[Winnie-the-Pooh, The House at Pooh Corner, Th...",6
1,A_G_Howard,4.020000,"[Splintered (Splintered, #1)]",1
2,A_J_Cronin,4.220000,[The Keys of the Kingdom],1
3,A_J_Jacobs,3.750000,[The Year of Living Biblically],1
4,A_J_Salt,4.940000,[Nik Nassa & the Mark of Destiny],1
...,...,...,...,...
2640,_,3.988095,"[Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...",42
2641,_gota_Krist_f,4.340000,"[The Notebook, The Proof, The Third Lie]",1
2642,_mile_Zola,3.990000,"[Germinal (Les Rougon-Macquart, #13), L'Assomm...",4
2643,_ric_Emmanuel_Schmitt,4.160000,[Oscar et la dame rose],1


In [58]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]

author                                            Stephen_King
rating                                                 3.91875
name         [The Stand, The Shining (The Shining #1), It, ...
num_books                                                   56
Name: 2349, dtype: object

In [59]:
authors[authors.author=='Terry_Pratchett']

,author,rating,name,num_books
2429,Terry_Pratchett,4.103415,"[Good Omens, Night Watch (Discworld, #29), Sma...",41


#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [60]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

,rating,rating_count,review_count,year
count,5993.000000,5.993000e+03,5993.000000,5993.000000
mean,4.041997,5.118390e+04,2374.331220,1969.085099
std,0.260509,1.376493e+05,5493.093328,185.383169
min,2.000000,5.000000e+00,0.000000,-1500.000000
25%,3.870000,7.527000e+03,390.000000,1980.000000
50%,4.050000,1.807200e+04,936.000000,2002.000000
75%,4.210000,4.294300e+04,2212.000000,2009.000000
max,5.000000,2.958974e+06,136455.000000,2014.000000


You can also access a `groupby` dictionary style.

In [61]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

ValueError: too many values to unpack (expected 2)

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [ ]:
#Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
#We can then iterate over these subsets
# your code here
for year, subset in df.groupby('year'):
    #Find the best book of the year

    bestbook = subset[subset.rating == subset.rating.max()]
    if bestbook.shape[0] > 1:
        print(year, bestbook.name.values, bestbook.rating.values)
    else:
        print(year, bestbook.name.values[0], bestbook.rating.values[0])